In [2]:
import time
import math
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# ----------------------------
# 1) LOAD & PREP DATA
# ----------------------------
df = pd.read_csv("DeliveryDataset2.csv").reset_index(drop=True)
# name customers C1…C|C|
df["CustomerID"] = ["C"+str(i+1) for i in df.index]
customers = df["CustomerID"].tolist()
C = len(customers)

# drivers D1…D3
drivers = ["D1","D2","D3"]
D = len(drivers)

# depot is the first row
hub_lat, hub_lon = df.loc[0, ["lat","lon"]]

# demands
demand = dict(zip(df["CustomerID"], df["TotalLoad"]))
cap = {k:50 for k in drivers}

# T_max and shift time (minutes)
T_max     = 30
shift_min = 8*60

# service time at each customer (fixed)
s = {i: 5 for i in customers}  # 5 min to serve

# ----------------------------
# 2) DISTANCE & TRAVEL TIMES
# ----------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R*c

# build node list = depot '0' + customers
nodes = ["0"] + customers

# coords
coords = {"0":(hub_lat,hub_lon)}
for i,row in df.iterrows():
    coords[row.CustomerID] = (row.lat, row.lon)

# compute dist (km) and time (min) assuming avg speed=40 km/h
speed = 40
dist = {}
time_ = {}
for u in nodes:
    for v in nodes:
        if u==v: 
            dist[u,v]=0; time_[u,v]=0
        else:
            lat1,lon1 = coords[u]
            lat2,lon2 = coords[v]
            d = haversine(lat1,lon1,lat2,lon2)
            dist[u,v]  = d
            time_[u,v] = d/speed*60

# ----------------------------
# 3) BUILD Gurobi MODEL
# ----------------------------
m = gp.Model("Delivery")

# 3.1 VARIABLES
y = m.addVars(drivers, customers, vtype=GRB.BINARY, name="y") 
x = m.addVars(drivers, nodes, nodes, vtype=GRB.BINARY, name="x")
uvar = m.addVars(drivers, customers, lb=1, ub=C, vtype=GRB.CONTINUOUS, name="u")
t    = m.addVars(nodes, lb=0, ub=T_max, vtype=GRB.CONTINUOUS, name="t")
l    = m.addVars(drivers, nodes, lb=0, ub=50, vtype=GRB.CONTINUOUS, name="l")

# 3.2 PARAMETERS for objective weights
α_dh, α_df, α_cf, α_p = 0.25, 0.25, 0.25, 0.25

# precompute µ_pd and µ_ds placeholders (we linearize std dev via second-moment)
# for simplicity we'll compute these *after* solution to evaluate the multi-objective

# ----------------------------
# 4) OBJECTIVE
# ----------------------------
# Term 1: deadheading = sum over k, sum_{legs in route} dis
dead_head = gp.quicksum(x[k,u,v]*dist[u,v] 
                       for k in drivers for u in nodes for v in nodes)

# Term 4: priority reward = sum_{k,i} y[k,i] * dist(prev→i) * p_i / ds_i 
#   (we’ll approximate prev→i by hub→i here for linearity)
#    let p_i = demand[i]  as a stand-in
prio = gp.quicksum(y[k,i]*demand[i]/s[i] for k in drivers for i in customers)

obj = ( - α_dh*dead_head
       + α_df*0   # placeholder for p/d variance (nonlinear)
       + α_cf*0   # placeholder for wait–distance variance
       - α_p*prio
     )
m.setObjective(obj, GRB.MINIMIZE)

# ----------------------------
# 5) CONSTRAINTS
# ----------------------------

M = 1e4  # big-M

# 5.1 assignment: each customer served exactly once
for i in customers:
    m.addConstr(gp.quicksum(y[k,i] for k in drivers)==1)

# 5.2 service count = |C|
m.addConstr(gp.quicksum(y[k,i] for k in drivers for i in customers)==C)

# 5.3 sub-tours (MTZ)
for k in drivers:
    for i in customers:
      for j in customers:
        if i!=j:
            m.addConstr(uvar[k,i] - uvar[k,j] + C*x[k,i,j] <= C-1)

# 5.4 sequencing & wait-time caps
for k in drivers:
    for i in nodes:
      for j in customers:
        # t_j >= t_i + s_i + time[i,j] - M*(1 - x[k,i,j])
        si = 0 if i=="0" else s[i]
        m.addConstr(t[j] 
                    >= t[i] + si + time_[i,j] 
                      - M*(1 - x[k,i,j]))

for i in customers:
    m.addConstr(t[i] <= T_max)
# depot start
m.addConstr(t["0"] == 0)

# 5.5 capacity (big-M)
for k in drivers:
    # start empty
    m.addConstr(l[k,"0"] == 0)
    for i in nodes:
      for j in customers:
        if i!=j:
            dij = demand[j] if j!="0" else 0
            m.addConstr(l[k,j] 
                        >= l[k,i] + dij*y[k,j] 
                          - M*(1 - x[k,i,j]))
    for i in nodes:
        m.addConstr(l[k,i] <= cap[k])

# 5.6 shift-length
for k in drivers:
    total_travel = gp.quicksum(time_[i,j]*x[k,i,j] 
                               for i in nodes for j in nodes)
    total_service= gp.quicksum(s[i]*y[k,i] for i in customers)
    m.addConstr(total_travel + total_service <= shift_min)

# 5.7 flow-balance: if driver k serves i then they must leave i once
for k in drivers:
    for i in nodes:
        m.addConstr(gp.quicksum(x[k,i,j] for j in nodes if j!=i) 
                    == gp.quicksum(x[k,j,i] for j in nodes if j!=i))

# ----------------------------
# 6) SOLVE & REPORT
# ----------------------------

start = time.time()
m.optimize()
print("Elapsed:", time.time()-start)

# collect assignment
assign = {k:[] for k in drivers}
for k in drivers:
  for i in customers:
    if y[k,i].X > 0.5:
      assign[k].append(i)

print("\nAssignment:")
for k in drivers:
    print(f" {k}: {assign[k]}")

# final deadheading
dh_val = dead_head.getValue()
prio_val = prio.getValue()
print(f"\nDeadheading = {dh_val:.2f}, Priority = {prio_val:.2f}")

# You can now compute the exact driver-fairness and customer-fairness off-line
# from the solution arrays if needed.

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [3]:
import time, math
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# ----------------------------------------
# 1) LOAD & PREPARE DATA
# ----------------------------------------
df = pd.read_csv("DeliveryDataset3.csv")
df["CustomerID"] = [f"C{i+1}" for i in df.index]
customers = df["CustomerID"].tolist()
C = len(customers)

# allow up to C drivers
drivers = [f"D{i+1}" for i in range(C)]
D = len(drivers)

# depot coordinates = first data point
hub_lat, hub_lon = df.loc[0, ["lat","lon"]]

# parameters per customer
demand    = dict(zip(df.CustomerID, df.TotalLoad))
priority  = dict(zip(df.CustomerID,
                    df.Priority    if "Priority"   in df.columns else 
                    np.random.randint(1,6, size=C)))
wait_time = dict(zip(df.CustomerID,
                    df.WaitTime   if "WaitTime"   in df.columns else
                    np.random.randint(5,31, size=C)))

T_max      = 30      # max wait per customer (min)
service_tm = 5       # service time per customer (min)
shift_min  = 8*60    # driver shift length (min)
capacity   = 50      # kg per driver

# ----------------------------------------
# 2) PRECOMPUTE DISTANCES & TIMES
# ----------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

nodes = ["0"] + customers
coords = {"0":(hub_lat,hub_lon)}
for _,r in df.iterrows():
    coords[r.CustomerID] = (r.lat, r.lon)

speed = 40.0  # km/h
dist = {}
time_ = {}
for u in nodes:
    for v in nodes:
        if u==v:
            dist[u,v]  = 0.0
            time_[u,v] = 0.0
        else:
            la,lo = coords[u]
            lb,ln = coords[v]
            d = haversine(la,lo,lb,ln)
            dist[u,v]  = d
            time_[u,v] = d/speed*60.0

# ----------------------------------------
# 3) CREATE Gurobi MODEL
# ----------------------------------------
m = gp.Model("Delivery")

# 3.1 VARIABLES
x = m.addVars(drivers, nodes, nodes, vtype=GRB.BINARY, name="x")  # routing arcs
y = m.addVars(drivers, customers, vtype=GRB.BINARY, name="y")    # assignment
u = m.addVars(drivers, customers, lb=1, ub=C, vtype=GRB.CONTINUOUS, name="u")  # MTZ
t = m.addVars(nodes, lb=0, ub=T_max, vtype=GRB.CONTINUOUS, name="t")         # time
l = m.addVars(drivers, nodes, lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name="l")  # load

# 3.2 WEIGHTS & EPS
α_dh, α_df, α_cf, α_p = 0.25, 0.25, 0.25, 0.25
eps = 1e-3
M = 1e4

# ----------------------------------------
# 4) OBJECTIVE
# ----------------------------------------
# 1) deadheading
dead = gp.quicksum(x[k,u,v]*dist[u,v]
                   for k in drivers for u in nodes for v in nodes)

# 2) driver‐fairness var(Pr_k/Dh_k)
Pr = {k: gp.quicksum(x[k,i,j]*dist[i,j] for i in customers for j in customers)
      for k in drivers}
Dh = {k: gp.quicksum(x[k,'0',j]*dist['0',j] + x[k,j,'0']*dist[j,'0']
                     for j in customers)
      for k in drivers}
mu_pd = gp.quicksum(Pr[k]/(Dh[k]+eps) for k in drivers)/D
var_pd = gp.quicksum((Pr[k]/(Dh[k]+eps)-mu_pd)*(Pr[k]/(Dh[k]+eps)-mu_pd)
                     for k in drivers)/D

# 3) customer‐fairness var(t[i])
mu_ds = gp.quicksum(t[i] for i in customers)/C
var_ds = gp.quicksum((t[i]-mu_ds)*(t[i]-mu_ds) for i in customers)/C

# 4) priority reward
prio = gp.quicksum(x[k,i,j]*dist[i,j]*priority[j]/(t[j]+eps)
                   for k in drivers for i in nodes for j in customers)

m.setObjective( α_dh*dead
               + α_df*var_pd
               + α_cf*var_ds
               - α_p*prio,
               sense=GRB.MINIMIZE )

# ----------------------------------------
# 5) CONSTRAINTS
# ----------------------------------------
# 5.1 each customer served exactly once
for i in customers:
    m.addConstr(gp.quicksum(y[k,i] for k in drivers)==1)

# 5.2 no duplicate arcs from any node
for k in drivers:
    for i in nodes:
        m.addConstr(gp.quicksum(x[k,i,j] for j in nodes)<=1)

# 5.3 MTZ subtour elimination
for k in drivers:
    for i in customers:
        for j in customers:
            if i!=j:
                m.addConstr(u[k,i] - u[k,j] + C*x[k,i,j] <= C-1)

# 5.4 sequencing & wait‐time caps
for k in drivers:
    for i in nodes:
        for j in customers:
            if j=='0': continue
            si = 0 if i=='0' else service_tm
            m.addConstr( t[j]
                       >= t[i] + si + time_[i,j]
                         - M*(1-x[k,i,j]) )

# cap wait time
for i in customers:
    m.addConstr(t[i] <= T_max)
m.addConstr(t['0']==0)   # start at depot

# 5.5 capacity accumulation
for k in drivers:
    m.addConstr(l[k,'0']==0)
    for i in nodes:
        for j in customers:
            if j=='0': continue
            m.addConstr( l[k,j]
                       >= l[k,i] + demand[j]*x[k,i,j]
                         - M*(1-x[k,i,j]) )
    for i in nodes:
        m.addConstr(l[k,i] <= capacity)

# 5.6 shift‐length cap
for k in drivers:
    travel = gp.quicksum(time_[i,j]*x[k,i,j] for i in nodes for j in nodes)
    service = gp.quicksum(service_tm*y[k,i] for i in customers)
    m.addConstr(travel + service <= shift_min)

# 5.7 flow‐balance
for k in drivers:
    for i in nodes:
        m.addConstr( gp.quicksum(x[k,i,j] for j in nodes if j!=i)
                   ==gp.quicksum(x[k,j,i] for j in nodes if j!=i) )

# 5.8 link x & y: if driver visits j at all, y[k,j] must be 1
for k in drivers:
    for j in customers:
        m.addConstr( gp.quicksum(x[k,i,j] for i in nodes) <= y[k,j]*len(nodes) )
        m.addConstr( y[k,j] <= gp.quicksum(x[k,i,j] for i in nodes) )

# ----------------------------------------
# 6) SOLVE & REPORT
# ----------------------------------------
m.Params.TimeLimit = 300
start = time.time()
m.optimize()
print(f"Solve time: {time.time()-start:.1f}s")

# extract assignment
assignment = {k:[] for k in drivers}
for k in drivers:
    for j in customers:
        if y[k,j].X > 0.5:
            assignment[k].append(j)

# renumber used drivers
used = [k for k in drivers if assignment[k]]
for idx,k in enumerate(used,1):
    print(f"D{idx} → {sorted(assignment[k], key=lambda x:int(x[1:]))}")

GurobiError: Objective must be linear or quadratic

In [26]:
import time, math
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# ----------------------------------------
# 1) LOAD & PREPARE DATA
# ----------------------------------------
df = pd.read_csv("DeliveryDataset3.csv")
df["CustomerID"] = [f"C{i+1}" for i in df.index]
customers = df["CustomerID"].tolist()
C = len(customers)

drivers = [f"D{i+1}" for i in range(C)]
nodes   = ["0"] + customers

hub_lat, hub_lon = df.loc[0, ["lat","lon"]]
demand   = dict(zip(df.CustomerID, df.TotalLoad))
priority = dict(zip(df.CustomerID,
                   df.Priority    if "Priority" in df.columns else
                   np.random.randint(1,6, size=C)))
T_max, service_tm, shift_min, capacity = 30, 5, 8*60, 50

# ----------------------------------------
# 2) DISTANCES & TIMES
# ----------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

coords = {"0":(hub_lat,hub_lon)}
for _,r in df.iterrows():
    coords[r.CustomerID] = (r.lat, r.lon)

speed = 40.0
dist   = {}
time_  = {}
for u in nodes:
    for v in nodes:
        if u==v:
            dist[u,v], time_[u,v] = 0.0, 0.0
        else:
            la,lo = coords[u]
            lb,ln = coords[v]
            d = haversine(la,lo,lb,ln)
            dist[u,v]   = d
            time_[u,v]  = d/speed*60.0

# ----------------------------------------
# 3) BUILD Gurobi MODEL
# ----------------------------------------
m = gp.Model("Delivery")

# 3.1 VARIABLES
x = m.addVars(drivers, nodes, nodes, vtype=GRB.BINARY, name="x")
y = m.addVars(drivers, customers, vtype=GRB.BINARY, name="y")
u = m.addVars(drivers, customers, lb=1, ub=C, vtype=GRB.CONTINUOUS, name="u")
t = m.addVars(nodes, lb=0, ub=T_max, vtype=GRB.CONTINUOUS, name="t")
l = m.addVars(drivers, nodes, lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name="l")

# 3.2 FAIRNESS AUX VARS
# Pr_k = served-distance, Dh_k = empty-distance
pr  = m.addVars(drivers, lb=0.0, name="Pr")
dh  = m.addVars(drivers, lb=0.0, name="Dh")
z   = m.addVars(drivers, lb=0.0, name="fair_dev")  # >= |Pr-Dh|

# 3.3 PARAMETERS
α_dh, α_df, α_cf, α_p = 0.25, 0.25, 0.25, 0.25
M = 1e4

# ----------------------------------------
# 4) LINEARIZED OBJECTIVE TERMS
# ----------------------------------------
# 4.1 deadheading
dead = gp.quicksum(x[k,u_,v]*dist[u_,v]
                   for k in drivers
                   for u_ in nodes for v in nodes)

# 4.2 link pr and dh
for k in drivers:
    # served-distance = sum over served arcs between customers
    m.addConstr(pr[k] == gp.quicksum(x[k,i,j]*dist[i,j]
                                     for i in customers for j in customers))
    # empty-distance = outbound+return legs to depot
    m.addConstr(dh[k] == gp.quicksum(x[k,'0',j]*dist['0',j] + x[k,j,'0']*dist[j,'0']
                                     for j in customers))
    # |Pr-Dh| <= z,  and  -(Pr-Dh) <= z
    m.addConstr(pr[k] - dh[k] <= z[k])
    m.addConstr(dh[k] - pr[k] <= z[k])

# 4.3 customer-fairness surrogate = total start-time
cust_wait = gp.quicksum(t[i] for i in customers)

# 4.4 priority reward (linear)
prio = gp.quicksum(x[k,i,j]*dist[i,j]*priority[j]
                   for k in drivers for i in nodes for j in customers)

m.setObjective( α_dh*dead
               + α_df*gp.quicksum(z[k] for k in drivers)   # sum abs deviations
               + α_cf*cust_wait
               - α_p*prio,
               sense=GRB.MINIMIZE )

# ----------------------------------------
# 5) CONSTRAINTS
# ----------------------------------------
# 5.1 each customer exactly once
for i in customers:
    m.addConstr(gp.quicksum(y[k,i] for k in drivers)==1)

# 5.2 arc‐duplication
for k in drivers:
    for i in nodes:
        m.addConstr(gp.quicksum(x[k,i,j] for j in nodes)<=1)

# 5.3 MTZ subtours
for k in drivers:
    for i in customers:
        for j in customers:
            if i!=j:
                m.addConstr(u[k,i] - u[k,j] + C*x[k,i,j] <= C-1)

# 5.4 sequencing + wait‐time
for k in drivers:
    for i in nodes:
        for j in customers:
            si = 0 if i=='0' else service_tm
            m.addConstr( t[j]
                       >= t[i] + si + time_[i,j]
                         - M*(1-x[k,i,j]) )
for i in customers:
    m.addConstr(t[i] <= T_max)
m.addConstr(t['0']==0)

# 5.5 capacity accumulation
for k in drivers:
    m.addConstr(l[k,'0']==0)
    for i in nodes:
        for j in customers:
            m.addConstr(l[k,j]
                       >= l[k,i] + demand[j]*x[k,i,j]
                         - M*(1-x[k,i,j]))
    for i in nodes:
        m.addConstr(l[k,i] <= capacity)

# 5.6 shift‐length
for k in drivers:
    travel = gp.quicksum(time_[i,j]*x[k,i,j] for i in nodes for j in nodes)
    serv   = gp.quicksum(service_tm*y[k,i]   for i in customers)
    m.addConstr(travel + serv <= shift_min)

# 5.7 flow‐balance
for k in drivers:
    for i in nodes:
        m.addConstr( gp.quicksum(x[k,i,j] for j in nodes if j!=i)
                   ==gp.quicksum(x[k,j,i] for j in nodes if j!=i) )

# 5.8 link x↔y
for k in drivers:
    for j in customers:
        m.addConstr(gp.quicksum(x[k,i,j] for i in nodes) <= y[k,j]*len(nodes))
        m.addConstr(y[k,j] <= gp.quicksum(x[k,i,j] for i in nodes))

# ----------------------------------------
# 6) SOLVE & REPORT
# ----------------------------------------
m.Params.TimeLimit = 300
start = time.time()
m.optimize()
print(f"Total optimized cost: {m.ObjVal:.4f}\n")
print(f"Solve time: {time.time()-start:.1f}s\n")

# extract & renumber used drivers
assign = {k:[] for k in drivers}
for k in drivers:
    for j in customers:
        if y[k,j].X > 0.5:
            assign[k].append(j)

used = [k for k in drivers if assign[k]]
for idx,k in enumerate(used,1):
    print(f"D{idx} → {sorted(assign[k], key=lambda x:int(x[1:]))}")

Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 581 rows, 281 columns and 2341 nonzeros
Model fingerprint: 0xc8efaa41
Variable types: 76 continuous, 205 integer (205 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+04]
  Objective range  [3e-01, 5e+00]
  Bounds range     [1e+00, 5e+01]
  RHS range        [1e+00, 1e+04]
Presolve removed 101 rows and 36 columns
Presolve time: 0.01s
Presolved: 480 rows, 245 columns, 3115 nonzeros
Variable types: 70 continuous, 175 integer (175 binary)
Found heuristic solution: objective 20.7819140

Root relaxation: objective -9.225018e+00, 281 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bou

In [23]:
import time
import pandas as pd
import numpy as np
from pyomo.environ import *
from math import radians, sin, cos, sqrt, atan2

# 1) LOAD DATA
data = pd.read_csv("DeliveryDataset3.csv")
data["CustomerID"] = ["C" + str(i+1) for i in range(len(data))]
customers = data["CustomerID"].tolist()

# 2) DRIVERS
C = len(customers)
drivers = [f"D{i+1}" for i in range(C)]

# 3) HUB COORDINATES
hub_lat, hub_lon = data.loc[0, ["lat","lon"]]

# 4) DISTANCES
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

distance = {}
for d in drivers:
    for _, row in data.iterrows():
        c = row["CustomerID"]
        distance[d,c] = haversine(hub_lat, hub_lon, row.lat, row.lon)

# 5) PRIORITY & WAIT
if "Priority" in data.columns:
    priority = dict(zip(data.CustomerID, data.Priority))
else:
    priority = dict(zip(data.CustomerID, np.random.randint(1,5,size=len(data))))
if "WaitTime" in data.columns:
    wait_time = dict(zip(data.CustomerID, data.WaitTime))
else:
    wait_time = dict(zip(data.CustomerID, np.random.randint(5,30,size=len(data))))
T_max = 30

mean_dist = np.mean(list(distance.values()))
mean_wait = np.mean(list(wait_time.values()))

# 6) MODEL
model = ConcreteModel()
model.D = Set(initialize=drivers)
model.C = Set(initialize=customers)

# decision var: x[d,c] = 1 if driver d serves c
model.x = Var(model.D, model.C, domain=Binary)

# weights (mutable if you want to tune)
model.alpha_dh = Param(initialize=0.25, mutable=True)
model.alpha_d  = Param(initialize=0.25, mutable=True)
model.alpha_c  = Param(initialize=0.25, mutable=True)
model.alpha_p  = Param(initialize=0.25, mutable=True)

# 7) OBJECTIVE
def obj_rule(m):
    # deadheading
    dead = sum(m.x[d,c]*distance[d,c] for d in m.D for c in m.C)
    # driver‐fairness (variance of total distances per driver)
    dfair = sum(
        (sum(m.x[d,c]*distance[d,c] for c in m.C) - mean_dist)**2
        for d in m.D
    )/len(m.D)
    # customer fairness (constant wrt x)
    cfair = sum((wait_time[c]-mean_wait)**2 for c in m.C)/len(m.C)
    # priority factor
    prio = sum(m.x[d,c]*priority[c]/wait_time[c] for d in m.D for c in m.C)
    return ( m.alpha_dh*dead
           + m.alpha_d *dfair
           + m.alpha_c *cfair
           - m.alpha_p *prio )
model.obj = Objective(rule=obj_rule, sense=minimize)

# 8) CONSTRAINTS
# each customer → exactly one driver
model.one_per_customer = Constraint(
    model.C,
    rule=lambda m,c: sum(m.x[d,c] for d in m.D) == 1
)
# each driver must serve ≥1 customer
model.at_least_one = Constraint(
    model.D,
    rule=lambda m,d: sum(m.x[d,c] for c in m.C) >= 1
)
# total assignments = #customers
model.total_assign = Constraint(
    expr=sum(model.x[d,c] for d in model.D for c in model.C) == len(customers)
)
# wait‐time constraint
model.wait_cap = Constraint(
    model.C,
    rule=lambda m,c: wait_time[c] * sum(m.x[d,c] for d in m.D) <= T_max
)
# weights sum to 1
model.weight_sum = Constraint(
    expr=model.alpha_dh + model.alpha_d + model.alpha_c + model.alpha_p == 1
)

# 9) SOLVE
solver = SolverFactory('gurobi', executable='C:\\Users\\DELL\\AMPL\\gurobi.exe')
start = time.time()
solver.solve(model, tee=True)
print(f"\nSolve time: {time.time()-start:.2f}s")

# 10) EXTRACT & REPORT
assignment = {d:[] for d in drivers}
for d in model.D:
    for c in model.C:
        if model.x[d,c].value > 0.5:
            assignment[d].append(c)

print("\nBest assignment:")
for d in sorted(assignment):
    print(f"  {d}: {sorted(assignment[d], key=lambda x:int(x[1:]))}")

# compute final cost breakdown
dead_cost = sum(distance[d,c]
                for d in drivers for c in assignment[d])
driver_loads = [len(assignment[d]) for d in drivers]
driver_fairness_val   = np.var(driver_loads)
customer_fairness_val = np.var([wait_time[c] for c in customers])
priority_val          = sum(priority[c]/wait_time[c]
                            for d in drivers for c in assignment[d])
α_dh = model.alpha_dh.value
α_d  = model.alpha_d.value
α_c  = model.alpha_c.value
α_p  = model.alpha_p.value
final_cost = (α_dh*dead_cost
            + α_d*driver_fairness_val
            + α_c*customer_fairness_val
            - α_p*priority_val)

print("\nCost breakdown:")
print(f"  Deadheading       = {dead_cost:.4f}")
print(f"  Driver‐fairness   = {driver_fairness_val:.4f}")
print(f"  Customer‐fairness = {customer_fairness_val:.4f}")
print(f"  Priority factor   = {priority_val:.4f}")
print(f"  Overall objective = {final_cost:.4f}")


Read LP format model from file C:\Users\DELL\AppData\Local\Temp\tmp57io_hlg.pyomo.lp
Reading time = 0.02 seconds
x1: 32 rows, 101 columns, 400 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 32 rows, 101 columns and 400 nonzeros
Model fingerprint: 0xdf5c5b9e
Model has 450 quadratic objective terms
Variable types: 1 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-03, 2e+01]
  QObjective range [8e-02, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 12 rows and 1 columns
Presolve time: 0.02s
Presolved: 380 rows, 460 columns, 1280 nonzeros
Variable types: 0 continuous, 460 integer (460 binary)
Found heuristic solution: objective 30.5506660

